## Rigid body 6 DOF
Devlop a system for a rigid bofy in 6 DOF and do a simualtion

In [1]:
import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sympy as sp
import sympy.physics.mechanics as me
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import rigidbody
from rigidbody.substitute_dynamic_symbols import substitute_dynamic_symbols, find_name, find_derivative_name, lambdify, find_derivatives
from rigidbody.plotting import TrackPlot3dWidget,track_plot

from pydy.codegen.ode_function_generators import generate_ode_function
from scipy.integrate import odeint



In [3]:
x0,y0,z0 = me.dynamicsymbols('x0 y0 z0')
x01d,y01d,z01d = me.dynamicsymbols('x01d y01d z01d')
u,v,w = me.dynamicsymbols('u v w')

phi,theta,psi = me.dynamicsymbols('phi theta psi')

phi1d,theta1d,psi1d = me.dynamicsymbols('phi1d theta1d psi1d')

In [4]:
N = me.ReferenceFrame('N')

In [5]:
S = N.orientnew('S', 'Body', [psi,theta,phi],'ZYX')

In [6]:
S.ang_vel_in(N)

(-sin(theta)*psi' + phi')*S.x + (sin(phi)*cos(theta)*psi' + cos(phi)*theta')*S.y + (-sin(phi)*theta' + cos(phi)*cos(theta)*psi')*S.z

In [7]:
S.ang_acc_in(N)

(-sin(theta)*psi'' - cos(theta)*psi'*theta' + phi'')*S.x + (-sin(phi)*sin(theta)*psi'*theta' + sin(phi)*cos(theta)*psi'' - sin(phi)*phi'*theta' + cos(phi)*cos(theta)*phi'*psi' + cos(phi)*theta'')*S.y + (-sin(phi)*cos(theta)*phi'*psi' - sin(phi)*theta'' - sin(theta)*cos(phi)*psi'*theta' + cos(phi)*cos(theta)*psi'' - cos(phi)*phi'*theta')*S.z

In [8]:
M = me.Point('M')
O = M.locatenew('P',0)

M.set_vel(N,0)
O.set_vel(S,u*S.x + v*S.y + w*S.z)

O.v1pt_theory(M,N,S)

u*S.x + v*S.y + w*S.z

In [9]:
velocity_matrix = O.vel(N).to_matrix(N)
velocity_matrix

Matrix([
[ (sin(phi(t))*sin(psi(t)) + sin(theta(t))*cos(phi(t))*cos(psi(t)))*w(t) + (sin(phi(t))*sin(theta(t))*cos(psi(t)) - sin(psi(t))*cos(phi(t)))*v(t) + u(t)*cos(psi(t))*cos(theta(t))],
[(-sin(phi(t))*cos(psi(t)) + sin(psi(t))*sin(theta(t))*cos(phi(t)))*w(t) + (sin(phi(t))*sin(psi(t))*sin(theta(t)) + cos(phi(t))*cos(psi(t)))*v(t) + u(t)*sin(psi(t))*cos(theta(t))],
[                                                                                            -u(t)*sin(theta(t)) + v(t)*sin(phi(t))*cos(theta(t)) + w(t)*cos(phi(t))*cos(theta(t))]])

In [10]:
rotional_velocity_matrix = sp.simplify(S.ang_vel_in(N).to_matrix(N))
rotional_velocity_matrix

Matrix([
[-sin(psi(t))*Derivative(theta(t), t) + cos(psi(t))*cos(theta(t))*Derivative(phi(t), t)],
[ sin(psi(t))*cos(theta(t))*Derivative(phi(t), t) + cos(psi(t))*Derivative(theta(t), t)],
[                          -sin(theta(t))*Derivative(phi(t), t) + Derivative(psi(t), t)]])

## Mass

In [11]:
mass = sp.symbols('m')
#x_g, y_g, z_g = sp.symbols('x_g y_g z_g')

In [12]:
#c_og = me.Point('c_og')
#c_og.set_pos(O,x_g*S.x + y_g*S.y + z_g*S.z)
#c_og.set_vel(S,0)

In [13]:
#c_og.v2pt_theory(O,N,S)

## Inertia

In [14]:
I_xx, I_yy, I_zz = sp.symbols('I_xx, I_yy, I_zz')
body_inertia_dyadic = me.inertia(S, ixx=I_xx, iyy=I_yy, izz=I_zz)
body_inertia_dyadic

I_xx*(S.x|S.x) + I_yy*(S.y|S.y) + I_zz*(S.z|S.z)

In [15]:
body_inertia_dyadic.to_matrix(S)

Matrix([
[I_xx,    0,    0],
[   0, I_yy,    0],
[   0,    0, I_zz]])

In [16]:
body_inertia_dyadic.to_matrix(N)

Matrix([
[                                                                                                                                        I_xx*cos(psi(t))**2*cos(theta(t))**2 + I_yy*(sin(phi(t))*sin(theta(t))*cos(psi(t)) - sin(psi(t))*cos(phi(t)))**2 + I_zz*(sin(phi(t))*sin(psi(t)) + sin(theta(t))*cos(phi(t))*cos(psi(t)))**2, I_xx*sin(psi(t))*cos(psi(t))*cos(theta(t))**2 + I_yy*(sin(phi(t))*sin(psi(t))*sin(theta(t)) + cos(phi(t))*cos(psi(t)))*(sin(phi(t))*sin(theta(t))*cos(psi(t)) - sin(psi(t))*cos(phi(t))) + I_zz*(sin(phi(t))*sin(psi(t)) + sin(theta(t))*cos(phi(t))*cos(psi(t)))*(-sin(phi(t))*cos(psi(t)) + sin(psi(t))*sin(theta(t))*cos(phi(t))),  -I_xx*sin(theta(t))*cos(psi(t))*cos(theta(t)) + I_yy*(sin(phi(t))*sin(theta(t))*cos(psi(t)) - sin(psi(t))*cos(phi(t)))*sin(phi(t))*cos(theta(t)) + I_zz*(sin(phi(t))*sin(psi(t)) + sin(theta(t))*cos(phi(t))*cos(psi(t)))*cos(phi(t))*cos(theta(t))],
[I_xx*sin(psi(t))*cos(psi(t))*cos(theta(t))**2 + I_yy*(sin(phi(t))*sin(psi(t))*sin(theta(t)

In [17]:
body_central_inertia = (body_inertia_dyadic, O)

In [18]:
body = me.RigidBody('Rigid body', masscenter=O, frame = S,
                      mass=mass, inertia=body_central_inertia)

## Forces

In [19]:
fx, fy, fz, mx, my, mz = sp.symbols('f_x f_y f_z m_x m_y m_z')

In [20]:
force_vector = fx*S.x + fy*S.y + fz*S.z
torque_vector = mx*S.x + my*S.y + mz*S.z

In [21]:
force = (O, force_vector)
torque = (S, torque_vector)

## Equations of Motion

In [22]:
coordinates = [x0, y0, z0, phi, theta, psi]
speeds = [u, v, w, phi1d, theta1d, psi1d]

In [23]:
me.kinematic_equations(speeds[0:3],coordinates[0:3],'Body', rot_order='ZYX')

[-(v(t)*sin(z0(t)) + w(t)*cos(z0(t)))/cos(y0(t)) + Derivative(x0(t), t),
 -v(t)*cos(z0(t)) + w(t)*sin(z0(t)) + Derivative(y0(t), t),
 -(v(t)*sin(z0(t)) + w(t)*cos(z0(t)))*sin(y0(t))/cos(y0(t)) - u(t) + Derivative(z0(t), t)]

In [24]:
kinematical_differential_equations = [x0.diff() - velocity_matrix[0],                                      
                                      y0.diff() - velocity_matrix[1],
                                      z0.diff() - velocity_matrix[2],
                                      phi.diff() - phi1d,
                                      theta.diff() - theta1d,
                                      psi.diff() - psi1d,
                                     ]

In [25]:
sp.simplify(kinematical_differential_equations)

[-(sin(phi(t))*sin(psi(t)) + sin(theta(t))*cos(phi(t))*cos(psi(t)))*w(t) - (sin(phi(t))*sin(theta(t))*cos(psi(t)) - sin(psi(t))*cos(phi(t)))*v(t) - u(t)*cos(psi(t))*cos(theta(t)) + Derivative(x0(t), t),
 -(-sin(phi(t))*cos(psi(t)) + sin(psi(t))*sin(theta(t))*cos(phi(t)))*w(t) - (sin(phi(t))*sin(psi(t))*sin(theta(t)) + cos(phi(t))*cos(psi(t)))*v(t) - u(t)*sin(psi(t))*cos(theta(t)) + Derivative(y0(t), t),
 u(t)*sin(theta(t)) - v(t)*sin(phi(t))*cos(theta(t)) - w(t)*cos(phi(t))*cos(theta(t)) + Derivative(z0(t), t),
 -phi1d(t) + Derivative(phi(t), t),
 -theta1d(t) + Derivative(theta(t), t),
 -psi1d(t) + Derivative(psi(t), t)]

In [26]:
#?me.KanesMethod

In [27]:
kane = me.KanesMethod(N, coordinates, speeds, kinematical_differential_equations)

In [28]:
loads = [force,
         torque]

In [29]:
bodies = [body]
fr, frstar = kane.kanes_equations(bodies, loads)

In [30]:
constants = [I_xx, I_yy, I_zz,mass]

specified = [fx, fy, fz, mx, my, mz]  # External force/torque

right_hand_side = generate_ode_function(kane.forcing_full, coordinates,
                                        speeds, constants,
                                        mass_matrix=kane.mass_matrix_full,specifieds=specified)

In [31]:
coordinates_ = [0, 0, 0, 0, 0, 0]
speeds_ = [0, 0, 0, 0, 0, 0]

start = np.array(coordinates_+speeds_)

t_ = 0.
force_torque = [1,0,0,0,0,0]
numerical_specified = np.array(force_torque)

I_xx_ = 1
I_yy_ = 1
I_zz_ = 1
mass_ = 1

numerical_constants = np.array([I_xx_, I_yy_, I_zz_,mass_])

right_hand_side(start,t_,numerical_specified, numerical_constants)

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])

In [ ]:
def simulate(t,force_torque, I_xx,I_yy,I_zz,mass, initial_coordinates = [0, 0, 0, 0, 0, 0], 
             initial_speeds = [0, 0, 0, 0, 0, 0]):
    
    start = np.array(initial_coordinates+initial_speeds)
    
    numerical_specified = force_torque
        
    numerical_constants = np.array([I_xx, I_yy, I_zz, mass])
    
    df = pd.DataFrame(index=t)
    y = odeint(right_hand_side, start, t, args=(numerical_specified, numerical_constants))
    
    for i,symbol in enumerate(coordinates+speeds):
        name = symbol.name
        df[name] = y[:,i]
        
    return df

In [ ]:
t = np.linspace(0,10,100)

df = simulate(t=t, force_torque=[1,0,0,0,0,0],I_xx=1, I_yy=1, I_zz=1, mass=1)


fig,ax = plt.subplots()
df.plot(y='x0', ax=ax);
ax.set_xlabel('time [s]')

fig,ax = plt.subplots()
df.plot(y='u', ax=ax);
ax.set_xlabel('time [s]');

In [ ]:

radius = 10  # Radius of rotation [m]
w = 0.1  # Angle velocity [rad/s]
V = radius*w  # Speed of point [m/s]
t = np.linspace(0,2*np.pi/w,100)

mass=1

expected_acceleration = -radius*w**2
expected_force = mass*expected_acceleration


df = simulate(t=t, force_torque=[0,-expected_force,0,0,0,0],I_xx=1, I_yy=1, I_zz=1, mass=mass,
              initial_speeds=[V,0,0,0,0,w],initial_coordinates=[0, -radius, 0,0,0,0])


fig,ax = plt.subplots()
track_plot(df,ax, time_step = '5S')


fig,ax = plt.subplots()
df.plot(y='psi', ax=ax);
ax.set_xlabel('time')
ax.set_ylabel('psi')

In [ ]:
t = np.linspace(0,5,100)

df = simulate(t=t, force_torque=[1,2,0,0,0,0.2],I_xx=1, I_yy=1, I_zz=1, mass=1,
              initial_speeds=[1,0,0,0,0,0.0],initial_coordinates=[0, 0, 0, 0, 0, 0.0])


fig,ax = plt.subplots()
track_plot(df,ax)


fig,ax = plt.subplots()
df.plot(y='psi', ax=ax);
ax.set_xlabel('time')
ax.set_ylabel('psi')



In [ ]:
t = np.linspace(0,10,100)

l = 1
mass = 1
kzz = 0.3
I_zz = mass*(kzz*l)**2

df = simulate(t=t, force_torque=[0,0.9,0,0,0,0.0],I_xx=1, I_yy=I_zz, I_zz=I_zz, mass=mass,
              initial_speeds=[0,0,0,0,0,0.0],initial_coordinates=[0, 0, 0, 0, 0, np.pi])


fig,ax = plt.subplots()
track_plot(df,ax)


fig,ax = plt.subplots()
df.plot(y='psi', ax=ax);
ax.set_xlabel('time')
ax.set_ylabel('psi')

In [ ]:
t = np.linspace(0,10,100)

l = 1
mass = 1
kzz = 0.3
I_zz = mass*(kzz*l)**2

df = simulate(t=t, force_torque=[0,0.0,0,0,0,0.0],I_xx=1, I_yy=I_zz, I_zz=I_zz, mass=mass,
              initial_speeds=[0,1,0,0,0,0.0],initial_coordinates=[0, 0, 0, 0, 0, np.pi])


fig,ax = plt.subplots()
track_plot(df,ax)


fig,ax = plt.subplots()
df.plot(y='psi', ax=ax);
ax.set_xlabel('time')
ax.set_ylabel('psi')

In [ ]:

radius = 2  # Radius of rotation [m]
w = 0.1  # Angle velocity [rad/s]
V = radius*w  # Speed of point [m/s]
t = np.linspace(0,2*np.pi/w,1000)

mass=1

expected_acceleration = -radius*w**2
expected_force = mass*expected_acceleration


df = simulate(t=t, force_torque=[0,-expected_force,0.0,0,0,0],I_xx=1, I_yy=1, I_zz=1, mass=mass,
              initial_speeds=[V,0,0.1,0,0,w],initial_coordinates=[0, 0, 0,0,0,0])


In [ ]:
scene = TrackPlot3dWidget(df=df)

In [ ]:
t = np.linspace(0,2*np.pi/w,100)

mass = 1
I_xx = 10
I_yy = 10
I_zz = 10

t = np.linspace(0, 10, 100)


initial_speeds = [0,0,0,0.0,0.0,0.3]

initial_coordinates = [0, 0, 0, np.deg2rad(45), 0, 0]

force_torque = [0,0,0,0,0,0]

# Run an alternative simulation:
df = simulate(t=t, initial_speeds=initial_speeds, initial_coordinates=initial_coordinates,
                             mass=mass,I_xx=I_xx, I_yy=I_yy, I_zz=I_zz,force_torque=force_torque)

fig,ax = plt.subplots()
track_plot(df,ax, time_step = '5S')

fig,ax = plt.subplots()
df.plot(y='psi', ax=ax);
ax.set_xlabel('time')
ax.set_ylabel('psi')

In [ ]:
scene = TrackPlot3dWidget(df=df)

In [ ]:
t = np.linspace(0,2*np.pi/w,100)

mass = 1
I_xx = 10
I_yy = 10
I_zz = 10

t = np.linspace(0, 10, 100)


initial_speeds = [0,0,0,0.1,0.0,0.0]

initial_coordinates = [0, 0, 0, 0, 0, np.deg2rad(45)]

force_torque = [0,0,0,0,0,0]

# Run an alternative simulation:
df = simulate(t=t, initial_speeds=initial_speeds, initial_coordinates=initial_coordinates,
                             mass=mass,I_xx=I_xx, I_yy=I_yy, I_zz=I_zz,force_torque=force_torque)

scene = TrackPlot3dWidget(df=df)